## Combinators

Biological systems achieve much of their adaptability through the use of very general parts (cells) that are dynamically configured and consequently able to adjust as their environment changes. 

Computational systems usually do not use this strategy, instead relying on a hierarchy of custom parts and combinations. 

In recent years, large libraries of well-specified higher-level parts have raised the abstraction level of this activity. But the means of combination are rarely abstracted or shared, other than as "patterns".

In some situations we can improve on theis practice by simple strategies that promote the use of shared combination mechanisms. If the systems we build are made up from members of a family of "mix-and-match" components that combine to make new members of the family, perturbations of the requirements can sometimes be addressed by rearrangement of components.

A _system of combinators_ is a set of primitive parts and a set of means of combining parts such that the interface specifications of the combinations are the same as those of the primitives. This enables construction without accidental interactions between the parts.

Combinator systems provide a design strategy for domain-specific languages. The elements of the system are words in the language, and the combinators are used to combine them into phrases. Combinator systems have the significant advantage that they are easy to build and to reason about, but they have limitations, we which will discuss in _the next section_. When they fit the domain they are an excellent strategic choice.

But how do we arrange to buid our systems by combining elements of a family of mix-and-match components? We must identify a set of promitive components and a set of _combinators_ that combine components so as to make compound components with the same interface as the primitive components.

### Functional Combinators

The use of functional notation in mathematics is a combinator discipline. A function has a domain, from which its arguments are selected, and a range (or co-domain) of its possible values. There are combinators that produce new functions as combinations of others. For example the composition f &compfn; g of functions f and g is a new function that takes arguments in the domain of g and produces values in the codomain of f.

If two functions have the same domain and codomain, and if arithmetic is defined on their common codomain, then we can define the sum (or product) of the functions as the function that, when given an argument in their common domain, is the sum (or product) of the values of the two functions at that argument. Languages that allow first-class procedures provide a mechanism to support this means of combination, but what really matters is a good family of pieces.

Organizing a system around combinators has several advantages. The parts taht are made can be arbitrarily mixed and matched. Any combination yields a legal programme, whose behaviour transparently depends only on the behaviours of the parts and the way that they are combined. The context in whch a part appears does not change the behaviour of the part: it is always acceptable to pick up a component part to use it in a new context, without worry about its behaviour in that context. Thus such programmes are easy to write, easy to read, and easy to verify. _A programme built on combinators is extensible, because introduction of new parts or new combinators does not affect the behaviour of existing programmes._

We can think of function combinators as implementing wiring diagrams that specify how a function is built by combining its parts. For example, functional composition represents a box made of two sub-boxes so taht they output of the first feeds the input of the second. 

A programme that implements this idea is straightforward:

In [1]:
(defn compose [f g]
    (fn [& args]
        (f (apply g args))))

#'user/compose

In [2]:
((compose #(* % %) inc) 2)

9

(It gets more exciting if we want to check that the arities match: that the function represented by procedure f takes only one argument, to match the output of g. It gets even more fun if g can return multiple values and f must take those arguments. We may also want to check that the arguments passed to the composition are the right number for g. But these are fine points that awe will deal with later.)

We can demonstrate composition wiht a simple example:

In [3]:
((compose (fn [x] (list 'foo x))
          (fn [x] (list 'bar x)))
     'z)

(foo (bar z))

It is sometimes nicer to name the function that is being returned by a combinator. For example, we could write compose as:

In [4]:
(defn compose [f g]
    (let [the-composition (fn [& args] (f (apply g args)))]
        the-composition))

#'user/compose

The name the-composition is not defined outside of the scope of the definition of compose, so there is no obvious advantage to this way of writing the compose procedure. We often use anonymous procedures defined by lambda expressions in our programmes, as in the first version of compose above. So the choice of how to write the programme is mostly a matter of style.

Even with just this compose combinator we can write some rather elegant code. Consider the problem of computing the _n_th iterate of a function f<sup>n</sup>(x) = f(f<sup>n-1</sup>(x)). We can write this elegantly as a programme:

In [5]:
(defn my-iterate [n]
    (fn [f]
        (if (zero? n)
            identity
            (compose f ((my-iterate (dec n)) f)))))

#'user/my-iterate

In [6]:
(defn my-identity [x] x)

#'user/my-identity

In [7]:
(defn square [x] (* x x ))

#'user/square

The result of `((my-iterate n) f)` is a new function, of the same type as `f`. It can be used wherever f can be used. So (iterate n) is itself a function combinator. Now we can use this to determine the result of repeatedly squaring a number:

In [8]:
(((my-iterate 3) square) 5)

390625

Notice the analogy: function composition is like multiplication, so function iteration is like exponentiation.

There are many simple combinators that are generally useful in programming. We will present just a few here to give a feeling for the range of possibilities.

We can arrange to use two functions in paralle, then combine their results with a specied combined function. This parallel combination is implemented with the procedure 

In [9]:
(defn parallel-combine [h f g]
    (defn the-combination [& args]
        (h (apply f args) (apply g args)))
    the-combination)

#'user/parallel-combine

In [10]:
((parallel-combine list
                   (fn [x y z] (list 'foo x y z))
                   (fn [u v w] (list 'bar u v w)))
                    'a 'b 'c)

((foo a b c) (bar a b c))

The parallel-combine can be useful in organizing a complex process. For example, suppose we have a source of images of pieces of vegetables. We may have one procedure that given the image can estimate the colour of the vegetable, and another that can give a description of the shape (leaf, root, stalk, ...). We may have a third procedure that can combine these descriptions to identify the vegetable. These can be neatly composed with parallel-combine.

### Arity

The "tensor combinator" of two procedures is just a new procedure that takes a data structure combining arguments for the two procedures. It distributes those arguments to the two procedures, producing a daa structure that combines the values of the two procedures. The need to unbundle a data structure, operate on the parts separately, and rebundle the results is ubiquitous in programming. 


`spread-combine` is a generalization of the tensor product in multilinear algerbra. In the mathematical tensor product, `f` and `g` are linear functions of their inputs, and `h` is a trace over some shared indices; but tensors are just the special case that inspired this combinator.

The programme to implement `spread-combine` is a bit more complicated than `parallel-combine`, because it must distribute the correct argumnents to `f` and `g`. Here is a first draft of that code:

In [11]:
(declare get-arity)
(defn spread-combine [h f g]
     (let [n (get-arity f)]
         (defn the-combinatin [& args]
             (h (apply f (take n args))
                (apply g (drop n args))))
         the-combination))

#'user/spread-combine

This code requires a way of determining how many arguments a procedures takes (its _arity_), because it has to pick out the arguments for `f` and then pass the rest to `g`.

This version of `spread-combine` is not very good. The most egregious problem is that `the-combination` takes any number of arguments, so it does not have a well-edfined numerical arity, and thus is cannot be passed to another combinator that needs its arity. For example, the result of a `spread-combine` cannot be passed as the second argument `f` to another `spread-combine`. So, somehow, we have to decorate `the-combination` with an appropriate arity.

Here is a second draft:

In [12]:
(declare restrict-arity)
(defn spread-combine [h f g]
    (let [n (get-arity f) m (get-arity g)
          t (+ n m)]
        (defn the-combination [& args]
            (h (apply f (take n args))
               (apply g (drop n args))))
        (restrict-arity the-combination t)))

#'user/spread-combine

In [13]:
(defn restrict-arity [f nargs]
    (with-meta f {:arity nargs}))

#'user/restrict-arity

In [14]:
(defn get-arity [f]
    (-> f meta :arity))

#'user/get-arity

In [17]:
((spread-combine list
                 (restrict-arity (fn [x y] (list 'foo x y)) 2) 
                 (restrict-arity (fn [u v w] (list 'bar u v w)) 3))
                                                'a 'b 'c 'd 'e)

((foo a b) (bar c d e))

This arity restricting logic gives us a way to figure out which (/ how many) arguments we should pass to each function. 

There is a small "improvement" we could make.

In [21]:
(defn spread-combine [h f g]
    (let [n (get-arity f) m (get-arity g)
          t (+ n m)]
        (defn the-combination [& args]
            (h (apply f (take n args))
               (apply g (->> args (drop n) (take m)))))
        (restrict-arity the-combination t)))

#'user/spread-combine

In [23]:
((spread-combine list
                 (restrict-arity (fn [x y] (list 'foo x y)) 2) 
                 (restrict-arity (fn [u v w] (list 'bar u v w)) 3))
                                                'a 'b 'c 'd 'e)

((foo a b) (bar c d e))

What happens if we provide an extra argument?

In [24]:
((spread-combine list
                 (restrict-arity (fn [x y] (list 'foo x y)) 2) 
                 (restrict-arity (fn [u v w] (list 'bar u v w)) 3))
                                                'a 'b 'c 'd 'e 'f)

((foo a b) (bar c d e))

With the older version ...

In [25]:
(defn spread-combine [h f g]
    (let [n (get-arity f) m (get-arity g)
          t (+ n m)]
        (defn the-combination [& args]
            (h (apply f (take n args))
               (apply g (drop n args))))
        (restrict-arity the-combination t)))
((spread-combine list
                 (restrict-arity (fn [x y] (list 'foo x y)) 2) 
                 (restrict-arity (fn [u v w] (list 'bar u v w)) 3))
                                                'a 'b 'c 'd 'e 'f)

Execution error (ArityException) at user$spread_combine$the_combination__4187/doInvoke (REPL:6).
Wrong number of args (4) passed to: user/eval4190/fn--4193


class clojure.lang.ArityException: 

Actually this is probably good enough. Clojure gives us the arity checking we need, without the additional complexity described in SDF.